In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torchvision import transforms,datasets,models
import torch.nn.functional as F
from sklearn.metrics import confusion_matrix
from tqdm import tqdm
import pandas as pd

import urllib.request as request

In [ ]:
class config:
    def __init__(self):
        self.ROOT_DATA_DIR = "hymenoptera"
        self.EPOCH = 10
        self.BATCH_SIZE = 32
        self.LEARNING_RATE = 0.01
        self.IMAGE_SIZE  = (224 , 224)
        self.DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
        print(f"this notebook is using: {self.DEVICE}")
        self.SEED = 205 
      
    def create_dir(self,dir_path):
      os.makedirs(dir_path,exist_ok=True)
      print(f'{dir_path} directory is created')
config = config()

##Download Data

In [ ]:
data_url = 'https://download.pytorch.org/tutorial/hymenoptera_data.zip'

In [ ]:
config.create_dir(dir_path=config.ROOT_DATA_DIR)

data_zip_file = "data.zip"
data_file_path = os.path.join(config.ROOT_DATA_DIR,data_zip_file)

request.urlretrieve(data_url,data_file_path)

In [ ]:
from zipfile import ZipFile
def unzip_file(source: str, dest: str) -> None:
    print(f"extraction started...")

    with ZipFile(source, "r") as zip_f:
        zip_f.extractall(dest)
    print(f"extracted {source} to {dest}")

unzip_file(data_file_path,config.ROOT_DATA_DIR)

###Create Path

In [ ]:
from pathlib import Path
Path('/content/hymenoptera/hymenoptera_data/train')

In [ ]:
train_path = Path('/content/hymenoptera/hymenoptera_data/train')
test_path = Path('/content/hymenoptera/hymenoptera_data/val')

###Normalize the size of Image ,concept are used many places

Mean of  each channel images

for 28x28 image size

mean = sum(value of pixel)/784 {28x28}

std = 

(data-mean)/std

In [ ]:
mean = torch.tensor([0.5,0.5,0.5])
std = torch.tensor([0.5,0.5,0.5])

### Transform the image

In [ ]:

train_transform  = transforms.Compose([
                                        transforms.Resize(config.IMAGE_SIZE),
                                        transforms.RandomRotation(degrees=20),
                                        transforms.ToTensor(),
                                        transforms.Normalize(mean,std)

])

test_transform  = transforms.Compose([
                                          transforms.Resize(config.IMAGE_SIZE),
                                          transforms.ToTensor(),
                                          transforms.Normalize(mean,std)

])



In [ ]:
# Featch Data From Local
train_data = datasets.ImageFolder(root=train_path,transform=train_transform)
test_data = datasets.ImageFolder(root=test_path,transform=test_transform)

In [ ]:
# Mapping Features
label_map = train_data.class_to_idx
label_map

In [ ]:
train_data  # Describe the data 

### Dataloader

In [ ]:
train_loader = DataLoader(
                               dataset= train_data,
                               batch_size = config.BATCH_SIZE,
                               shuffle=True
                                )

test_loader = DataLoader(
                               dataset= test_data,
                               batch_size = config.BATCH_SIZE,
                               shuffle=False
                                )

In [ ]:
data = next(iter(train_loader))
len(data)

In [ ]:
images,label = data

In [ ]:

images.shape , label.shape # 32 is batch_size , 3 is RGB Channel , 28x28 is image size -----label is 32 bcoz of batch_size

Visulzize the image

In [ ]:
img = images[0]
img.shape

In [ ]:
# (3,224,224) convert into ----------> (224,224,3)
plt.imshow(img.permute(1,2,0))

### Download use pre Trained Model for transfer learning (Alexnet)

In [ ]:
model = models.alexnet(pretrained=True)    #Imagenet dataset 1000 no of classification image will be use

In [ ]:
print(model)  # To See the Archetecture of models

Why we using this pretrained models?
if we have that model which already trained 1000 no of images i.e already seems so many features 

### Count No. of Trained Parameter 

In [ ]:
def count_params(model):
  model_params = {"Modules": list(), "Parameters": list()}
  total = {"trainable": 0, "non_trainable": 0} 
  for name, parameters in model.named_parameters():
    param = parameters.numel()
    if not parameters.requires_grad:
      total["non_trainable"] += param
      continue
    model_params["Modules"].append(name)
    model_params["Parameters"].append(param)
    total["trainable"] += param
  df = pd.DataFrame(model_params)
  df = df.style.set_caption(f"Total parameters: {total}")
  return df

count_params(model)

In [ ]:
### Convert Trainable to Non-Trainable

In [ ]:
# next(iter(model.parameters()))

In [ ]:
# (but we dont want make them trainable so this weight is update i.e they take a lot of time)
# So We Freeze all the model Layers

for parameters in model.parameters():
  parameters.requires_grad = False  # meaning of requires_grad is ready for back propogation,
                                    # that is calculating the gradient at every steps after that they go for optimizer step ,
                                    # it will do the gradient descent


In [ ]:
count_params(model)

### Create our own Classifier by using alexnet

In [ ]:
model.classifier

In [ ]:
model.classifier = nn.Sequential(
    nn.Linear(in_features=9216, out_features=100 , bias=True),
    nn.ReLU(inplace=True),
    nn.Dropout(p=0.5,inplace=False),
    nn.Linear(in_features=100,out_features=2,bias=True) # out_feature=2 bcoz we have two features
)

In [ ]:
print(model)

In [ ]:
count_params(model)

###Training Loop

In [ ]:
model.to(config.DEVICE)

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters())

In [ ]:
for epoch in range(config.EPOCH):
  with tqdm(train_loader) as tqdm_epoch:
    for images,label in tqdm_epoch:
      tqdm_epoch.set_description(f"Epoch {epoch +1}/{config.EPOCH}")

      #put the images on device
      images = images.to(config.DEVICE)
      label = label.to(config.DEVICE)

      #Forward pass
      outputs =model(images)
      loss = criterion(outputs,label)  #passing the pred and targets

       #Backward Pass 
      optimizer.zero_grad() # reset the weight/gradient to zero or past gradient
      loss.backward() #calculate the gradient
      optimizer.step() # weight update

      tqdm_epoch.set_postfix(loss=loss.item())

#it just applying on classifier part and  frozen above part(not be calculated any gradient for feature or above part)
# (classifier): Sequential(
#     (0): Linear(in_features=9216, out_features=100, bias=True)
#     (1): ReLU(inplace=True)
#     (2): Dropout(p=0.5, inplace=False)
#     (3): Linear(in_features=100, out_features=2, bias=True)

### Save the model

In [ ]:
os.makedirs('model_dir', exist_ok=True)
model_file = os.path.join('model_dir','CNN_model.pth')
torch.save(model,model_file)

### Evalute the model

In [ ]:
pred = np.array([])
target = np.array([])

with torch.no_grad():  #Now no gradient are update for prediction, all gradient are frozen
  for batch,data in enumerate(test_loader): # enmurate doing first show index/sequence value and second the data
    images = data[0].to(config.DEVICE)  
    label = data[1].to(config.DEVICE) 

    y_pred = model(images)


    
    pred = np.concatenate((pred , torch.argmax(y_pred,1).cpu().numpy()))
    target = np.concatenate((target,label.cpu().numpy()))

    

In [ ]:
cm = confusion_matrix(target,pred)
cm

In [ ]:
label_map

In [ ]:
plt.figure(figsize=(5,5))
sns.heatmap(cm , annot=True , fmt='d', xticklabels=label_map.keys() , yticklabels= label_map.keys(), cbar=False)

###Prediction

In [ ]:
data = next(iter(test_loader))
data

In [ ]:
len(data)

In [ ]:
images,labels = data

In [ ]:
images.shape

In [ ]:
img = images[0]
img

In [ ]:
idx = 2
img = images[idx]
img.shape
plt.imshow(img.permute(1,2,0))

In [ ]:
label = labels[idx]
label_map[label.item()]

In [ ]:
img.unsqueeze(dim=1)

In [ ]:
img_on_gpu = img.unsqueeze(0).to(config.DEVICE)

pred_prob = F.softmax(model(img_on_gpu),dim=1)
pred_prob


In [ ]:
argmax = torch.argmax(pred_prob).item()
argmax

In [ ]:
inv_label_map = {val : key for key,val in label_map.items()}
inv_label_map

In [ ]:
inv_label_map[argmax] , inv_label_map[labels[2].item()]

###Create prediction function

In [ ]:
def predict(data,model,label_map,device,idx=0):
  images,labels = data

  img = images[idx]
  label = labels[idx]

  plt.imshow(img.permute(1,2,0))
  img_on_gpu = img.unsqueeze(0).to(config.DEVICE)
  pred_prob = F.softmax(model(img_on_gpu),dim=1)
  argmax = torch.argmax(pred_prob).item()

  predicted_label = label_map[argmax]
  actual_label = label_map[label.item()]

  plt.title(f'actual : {actual_label} | predicted : {predicted_label}')
  plt.axis('off')
  return actual_label , predicted_label

In [ ]:
predict(data,model,inv_label_map,config.DEVICE,idx=2)